In [1]:
mydatadir = "/data1/peterr/bilingual_data_for_ABClf"
outdir = "/data1/peterr/bilingual_data_for_ABClf_output/tsv"
import os
import pandas as pd
from ABClf import load_lexicon, get_variant, count_variants, counts_to_category
cols = "URL_en    URL_is    sent_en    sent_is    bleualign_score    paragraph_id_en    paragraph_id_is    deferred_hash_en    deferred_hash_is          bifixer_hash    bifixer_score    bicleaner_ai_score".split()
lex = load_lexicon()


import parse
def get_domain(url:str) -> str:
    pattern = "{protocol}://{domain}/{rest}"
    p = parse.compile(pattern)
    modified_pattern = "{protocol}://{domain}/"
    mp = parse.compile(modified_pattern)
    try:
        parse_result = p.parse(url)
        domain = parse_result["domain"]
    except TypeError:
        parse_result = mp.parse(url)
        domain = parse_result["domain"]
    return domain



import sys
import csv

csv.field_size_limit(131072<<4)


131072

In [4]:

file = 'en-tr.deduped.txt'
lang = file[3:5]
print(lang)

endpath = os.path.join(
        outdir,
        file+".ABClf_output.txt",
    )
print(file)
df = pd.read_csv(
    os.path.join(
        mydatadir,
        file),
    sep="\t",
    engine="python",
    quoting=3,
    on_bad_lines="warn",
    ).dropna()

df.columns = [col.replace("_is", f"_{lang}") for col in cols]


s = df.groupby("URL_en")["sent_en"].apply(" ".join)
ndf = pd.DataFrame(s)
ndf["en_document_level_variant"] = ndf.sent_en.apply(lambda s: get_variant(s, lex=lex))
ndf["per_token_unbalanced"] = ndf.sent_en.apply(lambda s: count_variants(s, lex)[1])
ndf["domain_en"] = [get_domain(url) for url in ndf.index.values]
ndf["URL_en"] = ndf.index

df = pd.merge(
        df,
        ndf[['en_document_level_variant', 'per_token_unbalanced', 'domain_en',]],
        how="left",
        left_on="URL_en",
        right_index=True,
        sort=False,
        suffixes=("_x", "_y"),
        copy=True,
        indicator=False,
        validate=None,
        )
del ndf


tr
en-tr.deduped.txt


In [26]:

df["en_domain_level_variant"] = "UNK"
counts = df.domain_en.value_counts()
suitable_domains = counts[counts >= 3].index.values
l = len(suitable_domains)
pruned_df = df.loc[:, ["per_token_unbalanced", "domain_en"]].copy()
def get_variant(domain):
    subset = pruned_df.loc[pruned_df.domain_en == domain, :]
    d = dict()
    for line in subset.per_token_unbalanced.values:
        if not line:
            continue
        for word, payload in line.items():
            variant, count = payload.get("variant"), payload.get("count")
            d[variant] = d.get(variant, 0) + 1
    variant = counts_to_category(d)
    return variant

print("Ok, assigning variants...")
l = len(suitable_domains)
print("Nr of domains to process: ", l)


Ok, assigning variants...
Nr of domains to process:  63626


In [27]:
# from concurrent.futures import ProcessPoolExecutor, as_completed
# suitable_domains = counts[counts >= 3].index.values[:636]
# with ProcessPoolExecutor(max_workers=60) as executor:
#     variants = list(executor.map(get_variant, suitable_domains))
# print("Calculation finished. Assigning.")

gb = df.groupby("domain_en").en_document_level_variant.agg(lambda x: pd.Series.mode(x).iloc[0])


In [29]:
df = pd.merge(
        df,
        gb,
        how="left",
        left_on="domain_en",
        right_index=True,
        sort=False,

        )
        

In [34]:
#df["en_document_level_variant"] = df.en_document_level_variant_y
#df = df.drop(columns=["en_document_level_variant_y", "en_document_level_variant_x"])
print("Variant assigned, saving to ", endpath)
#df = df.drop(columns="per_token_unbalanced")
df.to_csv(
    endpath,
    index=False,
    sep="\t"
)

Variant assigned, saving to  /data1/peterr/bilingual_data_for_ABClf_output/tsv/en-tr.deduped.txt.ABClf_output.txt


60: 

In [31]:
df.head(3)

,URL_en,URL_tr,sent_en,sent_tr,bleualign_score,paragraph_id_en,paragraph_id_tr,deferred_hash_en,deferred_hash_tr,bifixer_hash,bifixer_score,bicleaner_ai_score,en_document_level_variant_x,domain_en,en_domain_level_variant,en_document_level_variant_y
0,https://www.booking.com/hotel/pt/apartments-ma...,https://www.booking.com/hotel/hr/casa-sandra-d...,Can I park a car at Apartments Madeira City Ce...,Apartments Casa Toni Dubrovnik tesisinde teras...,0.190693,p788s0,p638s0,353c42d4b0dbf2dc,9a1416ecb90d12c1,00000047caab863f,93.73,0.288,A,www.booking.com,UNK,A
1,https://avesis.acibadem.edu.tr/activitycitatio...,https://avesis.comu.edu.tr/activitycitation/in...,"@article{article, author={AYŞE SAĞDUYU KOCAMAN...","Kopyala @article{article, author={GÜLTEN GÜMÜŞ...",0.262317,p24s0+p24s1,p23s0+p24s0,478ba8b60b952259+d4d14ad9894005ae#1,aab6e1e426cb7342+ffd7bc8c6bb4d615#1,0000005c36321f6c,105.67,0.000,UNK,avesis.acibadem.edu.tr,UNK,UNK
2,https://arts.ozyegin.edu.tr/en/portfolio/bicycle,http://www.naybir.org/?product=2519869&pt=Ar%C...,"A sad boy on the right, neck and face leaning ...",Arı sokmasına karşı koruyan en önemli araçlard...,0.140880,p16s0,p39s0,d6a3ffa08fc6e0c9,273f7b825cc4a38c,0000007b37ac2248,100.51,0.021,UNK,arts.ozyegin.edu.tr,UNK,UNK


In [11]:
test = df.head(1000).groupby("domain_en").en_document_level_variant.agg(pd.Series.mode)

In [21]:
test.values[5]

'UNK'

In [25]:
test = df.head(1000).groupby("domain_en").en_document_level_variant.agg(lambda x: pd.Series.mode(x).iloc[0])

test

domain_en
7thshare.com                    UNK
abl.gtu.edu.tr                  UNK
acikerisim.aku.edu.tr             A
acikerisim.bartin.edu.tr          A
acikerisim.erdogan.edu.tr         A
                               ... 
yapimerkezi.com.tr                A
yenimedya.wordpress.com           B
yerel.siyasal.marmara.edu.tr    UNK
zaviews.medium.com              UNK
zeo.org                           A
Name: en_document_level_variant, Length: 616, dtype: object